# Description
---

This notebook can be used to reproduce results form the paper:
"Cross-column DFT-based QSRR model development powered by Machine Learning".

Data (molecular descriptors, column characteristics, and experimental retention times) are loaded, split into training, validation and blind test. QSRR models are built using four ML methods (Ridge Regression, Partial Least Squares, Random Forests and Gradient Boosting). Hyperparameters of the ML QSRR models are optimized using a grid search.

# Requirements
---

1. jupyter
2. notebook
3. numpy
4. pandas
5. matplotlib
6. seaborn
7. scikit-learn
8. shap

# Imports
---

In [1]:
# Install requirements
%pip install numpy==1.22 pandas matplotlib seaborn scikit-learn shap

     --------------------------------------- 14.7/14.7 MB 10.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.4
    Uninstalling numpy-1.24.4:
      Successfully uninstalled numpy-1.24.4
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\user4\\PycharmProjects\\cross_column_qsrr\\venv\\Lib\\site-packages\\~umpy\\.libs\\libopenblas64__v0.3.21-gcc_10_3_0.dll'
Check the permissions.


[notice] A new release of pip available: 22.2.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%load_ext autoreload
%autoreload 2

import os
import sys
from copy import deepcopy
from typing import (
    Any,
    Dict,
    List
)

import numpy as np
from numpy import ndarray

import pandas as pd
from pandas import DataFrame

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor
from sklearn.cross_decomposition import PLSRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import RidgeCV
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import (
    cross_val_score,
    KFold,
    GridSearchCV
)
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [3]:
# Place the package on the path

QSRR_PATH: str = os.path.join(os.path.dirname(os.getcwd()))

if QSRR_PATH not in sys.path:
    sys.path.insert(0, QSRR_PATH)

In [4]:
from qsrr.analysis import analyze_model
from qsrr.config import (
    plot_settings,
    project1_split_indices
)
from qsrr.visuals import Visualizer

plt.rcParams.update(plot_settings)

sns.set()

ImportError: cannot import name 'Exception' from 'qsrr.exceptions' (C:\Users\user4\PycharmProjects\cross_column_qsrr\qsrr\exceptions\__init__.py)

# Data Loading & Processing
---

In [ ]:
# Load Data
_data_df: DataFrame = pd.read_csv(
    '../data/2023-09-25-qsrr_metlin_dataset.csv'
)

# Drop columns= 'name' and '#', 'ID', "Length"
_data_df.drop(
    columns=['#','name', 'ID', "Length"],
    errors="ignore",
    inplace=True
)

# Display Data
display(_data_df.head())

# Data Analysis
---

# Train/Test Split

In [ ]:
# X- and y- data
_x: ndarray = _data_df.iloc[:,:-1].values  # Variables -> x1 to x15
_y: ndarray = _data_df.iloc[:,-1].values   # Target: -> tR : retention tiime

# Split initial data into training & blind test sets
_x_train_all, _x_bt, _y_train_all, _y_bt = train_test_split(
    _x,
    _y,
    test_size=0.3
    ,shuffle=True,
    random_state=12345  # For reproducibility
)

# Split the training data further into training and validation
_x_train, _x_validation, _y_train, _y_validation = train_test_split(
    _x_train_all,
    _y_train_all,
    test_size=0.3,
    random_state=12345   # For reproducibility
)

# Summary
print('X-train:', _x_train.shape)
print('y-train:', _y_train.shape)
print('- - '*10)
print('X-validation:', _x_validation.shape)
print('y-validation:', _y_validation.shape)
print('- - '*10)
print('X-BT:', _x_bt.shape)
print('y-BT:', _y_bt.shape)
print('- - '*10)

# Hyper-Parameter Optimization, QSRR model building & analysis
---

## Random Forests
---

In [ ]:
# RFs parameter grid
_rfs_param_grid: Dict[str, Any] = {
    'n_estimators': range(10, 210, 10),
    'max_features': [None, 'sqrt'],
    'max_depth': range(2, 30, 3),
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 3, 5],
    'bootstrap': [True, False]
}

In [ ]:
# Instantiate model
_rfs_model: GridSearchCV = GridSearchCV(
    estimator=RandomForestRegressor(random_state=12345),
    param_grid=_rfs_param_grid,
    scoring='neg_root_mean_squared_error',
    cv=KFold(3),
    n_jobs=20,
    verbose=2,
    refit=True
)

In [ ]:
# Fit model
_rfs_model.fit(
    deepcopy(_x_train),
    deepcopy(_y_train)
)

In [ ]:
# Optimal Results
print(f"Optimal hyper-parameters : {_rfs_model.best_params_}")
print(f"Optimal RMSE : {-_rfs_model.best_score_.round(3)}")

# Optimal Model
_rfs_optimal_model: RandomForestRegressor = _rfs_model.best_estimator_

display(_rfs_optimal_model)

In [ ]:
# Analyze results
_rf_predictions_df, _rf_metrics_df = analyze_model(
    model=_rfs_optimal_model,
    cv=ps,
    title="Random Forests",
    x_train=deepcopy(_x_train),
    y_train=deepcopy(_y_train.ravel()),
    x_validation=deepcopy(_x_validation),
    y_validation=deepcopy(_y_validation.ravel()),
    x_bt=deepcopy(_x_bt),
    y_bt=deepcopy(_y_bt.ravel()),
    x_train_all=deepcopy(_x_train_all),
    y_train_all=deepcopy(_y_train_all.ravel()),
    x_all=deepcopy(_x),
    y_all=deepcopy(_y.ravel()),
    column_names=np.array(list(_data_df.drop(columns=["rt"]).columns))
)

In [ ]:
display(_rf_predictions_df.head())
display(_rf_metrics_df.head())

##  Partial Least Squares
---

In [ ]:
# Number of PLS components
_n_latent_variables: ndarray = np.array(range(1, 14, 1))

# Optimization of n(LVs)

_rmsecvs: List[float] = []

for _n_lvs in _n_latent_variables:

    _pls_model = make_pipeline(StandardScaler(), PLSRegression(n_components=_n_lvs))

    _cv = KFold(n_splits=10, shuffle=True, random_state=12345)

    _score = cross_val_score(
        _pls_model,
        deepcopy(_x_train),
        deepcopy(_y_train),
        scoring='neg_root_mean_squared_error',
        cv=_cv
    )

    _rmse = -_score.mean()
    
    _rmsecvs.append(_rmse)

_rmsecvs: ndarray = np.array(_rmsecvs)

Visualizer.latent_variable_plot(
    rmsecvs=_rmsecvs,
    latent_variables=_n_latent_variables,
    optimal_n_lvs=6,  # For reproducibility
    y_max=0.51  # For reproducibility
)

In [ ]:
# Train PLS Model
_pls_optimal_model = make_pipeline(
    StandardScaler(),
    PLSRegression(n_components=6)
)

_pls_optimal_model.fit(
    deepcopy(_x_train),
    deepcopy(_y_train)
)

display(_pls_optimal_model)

In [ ]:
# Analyze results
_pls_predictions_df, _pls_metrics_df = analyze_model(
    model=_pls_optimal_model,
    cv=ps,
    title="PLS",
    x_train=deepcopy(_x_train),
    y_train=deepcopy(_y_train.ravel()),
    x_validation=deepcopy(_x_validation),
    y_validation=deepcopy(_y_validation.ravel()),
    x_bt=deepcopy(_x_bt),
    y_bt=deepcopy(_y_bt.ravel()),
    x_train_all=deepcopy(_x_train_all),
    y_train_all=deepcopy(_y_train_all.ravel()),
    x_all=deepcopy(_x),
    y_all=deepcopy(_y.ravel()),
    column_names=np.array(list(_data_df.drop(columns=["rt"]).columns)),
    b_plot_feature_importances=False
)

In [ ]:
# PLS Coefficient Plot
Visualizer.coefficient_plot(
    coefficients=_pls_optimal_model.named_steps["plsregression"].coef_,
    column_names=np.array(list(_data_df.drop(columns=["rt"])))
)

In [ ]:
display(_pls_predictions_df.head())
display(_pls_metrics_df)

## Gradient Boosting Regression
---

In [ ]:
# GB parameter grid
_gb_param_grid = {
    'n_estimators': np.arange(10, 210, 10),
    'learning_rate':[.001, 0.01, .1],
    'max_features': [None, 'sqrt'],
    'max_depth': range(1, 6, 1),
    'min_samples_split': [2, 5, 7],
    'min_samples_leaf': [1, 3, 5],
}

In [ ]:
# Instantiate GB model
_gb_model = GridSearchCV(
    estimator=GradientBoostingRegressor(random_state=12345),
    param_grid=_gb_param_grid,
    scoring='neg_root_mean_squared_error',
    cv=ps,
    n_jobs=6,
    verbose=1
)

In [ ]:
# Fit model
_gb_model.fit(
    deepcopy(_x_train_all),  # Use all, because we extracted the validation indices previously for reproducibility
    deepcopy(_y_train_all.ravel())   # Use all, because we extracted the validation indices previously for reproducibility
)

In [ ]:
# Optimal Results
print(f"Optimal hyper-parameters : {_gb_model.best_params_}")
print(f"Optimal RMSE : {-_gb_model.best_score_.round(3)}")

# Optimal Model
_gb_optimal_model: GradientBoostingRegressor = _gb_model.best_estimator_

display(_gb_optimal_model)

In [ ]:
# Analyze results
_gb_predictions_df, _gb_metrics_df = analyze_model(
    model=_gb_optimal_model,
    cv=ps,
    title="Gradient Boosting",
    x_train=deepcopy(_x_train),
    y_train=deepcopy(_y_train.ravel()),
    x_validation=deepcopy(_x_validation),
    y_validation=deepcopy(_y_validation.ravel()),
    x_bt=deepcopy(_x_bt),
    y_bt=deepcopy(_y_bt.ravel()),
    x_train_all=deepcopy(_x_train_all),
    y_train_all=deepcopy(_y_train_all.ravel()),
    x_all=deepcopy(_x),
    y_all=deepcopy(_y.ravel()),
    column_names=np.array(list(_data_df.drop(columns=["rt"]).columns)),
    b_plot_y_randomization=True
)

In [ ]:
display(_gb_predictions_df.head())
display(_gb_metrics_df.head())

## Ridge Regression
---

In [ ]:
# Optimize the regularization parameter (alpha)
_ridge_model = make_pipeline(
    StandardScaler(),
    RidgeCV(
        alphas=[0.01, 0.1, 1, 10, 50, 100, 200],
        scoring= 'neg_root_mean_squared_error',
        cv=ps
    )
)

In [ ]:
_ridge_model.fit(
    X=deepcopy(_x_train_all),
    y=deepcopy(_y_train_all.ravel())
)

In [ ]:
# Display Optimal Results
print('Optimal alpha :',  _ridge_model.named_steps['ridgecv'].alpha_)
print('Optimal RMSE : ',  -_ridge_model.named_steps['ridgecv'].best_score_)

In [ ]:
# Analyze results
_ridge_predictions_df, _ridge_metrics_df = analyze_model(
    model=_ridge_model,
    cv=ps,
    title="Ridge Regression",
    x_train=deepcopy(_x_train),
    y_train=deepcopy(_y_train.ravel()),
    x_validation=deepcopy(_x_validation),
    y_validation=deepcopy(_y_validation.ravel()),
    x_bt=deepcopy(_x_bt),
    y_bt=deepcopy(_y_bt.ravel()),
    x_train_all=deepcopy(_x_train_all),
    y_train_all=deepcopy(_y_train_all.ravel()),
    x_all=deepcopy(_x),
    y_all=deepcopy(_y.ravel()),
    column_names=np.array(list(_data_df.drop(columns=["TR"]).columns)),
    b_plot_feature_importances=False
)

In [ ]:
# Ridge Coefficient Plot
Visualizer.coefficient_plot(
    coefficients=_ridge_model.named_steps["ridgecv"].coef_,
    column_names=np.array(list(_data_df.drop(columns=["rt"])))
)

In [ ]:
display(_ridge_predictions_df.head())
display(_ridge_metrics_df.head())